In [ ]:
import numpy as np
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import pickle

# Number of sensors
M = 12

# Number of nodes
K = 6

# Locations of the nodes with respect to initial node
locations = (np.array([0,0]),np.array([0.45,0.99]),np.array([3.02,0.45]),np.array([5.61,0.90])
             ,np.array([8.03,1.46]),np.array([8.70,0.50]))

# Direction of arrivals
angles = np.array([-14,-30,-5])*2*(np.pi)/360
K_l = np.array([np.sin(angles), np.cos(angles)])

# Steering vector is implemented. It should have the shape: (12,3)
for j in range(0,len(locations)):
    for i in range(0,len(angles)):
        first_part = np.exp(1j*np.pi*locations[j].dot(K_l[:,i]))
        second_part = np.array([1, np.exp(1j*np.pi*1*np.sin(angles[i]))]) 
        A_k = np.transpose(np.array(first_part*second_part, ndmin=2))
        if i == 0:
            B_k = A_k
        else:
            B_k = np.concatenate((B_k, A_k), axis=1)
        
        if i == len(angles)-1:
            if j == 0:
                A = np.transpose(B_k)
            else:
                A = np.concatenate((A, np.transpose(B_k)), axis=1)

A = np.transpose(A)

MSE = np.zeros(100)

# uncomment here and comment for N_samples if you want to iterate over SNR
# MSE = np.zeros(80)
# for snr_dB in range(-20,60):

for N_samples in range(2,102):

    for i in range(500):

        snr_dB = 10
        
        # Signal(A*s) to noise(n) ratio
        received_snr = 10**(snr_dB/20)
        ratio_As_to_s = 1/2.19
        snr = received_snr*ratio_As_to_s
        #N_samples = 100

        # Source signal implementation (shape: (3,500))
        #signal = np.random.normal(0,np.sqrt(snr),(3,N_samples)) + 1j*np.random.normal(0,np.sqrt(snr),(3,N_samples))
        w = np.atleast_2d([np.pi/3, np.pi/4, np.pi/5]).T
        signal = (np.sqrt(snr/3))*np.exp(1j*w*(np.atleast_2d(np.arange(1,N_samples+1))))
        
        # Received signal power on sensors
        signal_power = sum(sum(np.abs(A.dot(signal))**2))/(12*N_samples)

        # Noise signal implementation (shape: (12,500))
        noise = np.random.normal(0,np.sqrt(0.5),(12,N_samples)) + 1j*np.random.normal(0,np.sqrt(0.5),(12,N_samples))
        
        if i == 0 and snr_dB == 0:
            print("SIGNAL TO NOISE RATIO")
            print(signal_power)

        # Received signal (shape: (12,500))
        z = A.dot(signal) + noise

        # Sample covariance matrix
        R_sample = z.dot(z.conj().T)

        # Eigenvalue and eigenvectors
        w_sample, v_sample = np.linalg.eig(R_sample)
        if i == 0 and snr_dB == -20:
            print()
            print("EIGENVALUES OF SAMPLE COVARIANCE MATRIX")
            print(w_sample[0])
            print(w_sample[1])
            print(w_sample[2])
            print(w_sample[3])

        # Sensor Selection Matrix (shape: (12,6))
        T = np.array([[1,0,0,0,0,0],
                      [1,0,0,0,0,0],
                      [0,1,0,0,0,0],
                      [0,1,0,0,0,0],
                      [0,0,1,0,0,0],
                      [0,0,1,0,0,0],
                      [0,0,0,1,0,0],
                      [0,0,0,1,0,0],
                      [0,0,0,0,1,0],
                      [0,0,0,0,1,0],
                      [0,0,0,0,0,1],
                      [0,0,0,0,0,1]])

        # Push-Sum Matrix (shape: (6,6))
        P = np.array([[0.2,0.2,0.2,0  ,0  ,0],
                      [0.2,0.2,0.2,0  ,0  ,0],
                      [0.6,0.6,0.2,0.2,0  ,0],
                      [0  ,0  ,0.4,0.2,0.2,0.2],
                      [0  ,0  ,0  ,0.2,0.2,0.2],
                      [0  ,0  ,0  ,0.4,0.6,0.6]])

        # Weight Vector  (shape: (6,1))
        w = np.atleast_2d([1,1,1,1,1,1]).T

        # Push-Sum Covariance Matrix Estimation
        R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P,30)).dot(T.T), R_sample)
        R_push_denominator = T.dot(np.linalg.matrix_power(P,30)).dot(w).dot(np.ones((1,6))).dot(T.T)

        # Push Sum Covariance Matrix (shape: (12,12))
        R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
        
        if i == 0 and snr_dB == -20:
            print()
            print("                            R_push")
            print("****************************************************************")
            print("****************************************************************")
            print(R_push)
            print()
            print("                           R_sample")
            print("****************************************************************")
            print("****************************************************************")
            print(R_sample)
            print()
            # Check Convergence Status
            print(R_push == R_sample)

        w_push, v_push = np.linalg.eig(R_push)

        # Upper group selection matrix J_up
        J_up = np.kron(np.eye(6),np.array([1,0]))

        # Lower group selection matrix J_down
        J_down = np.kron(np.eye(6),np.array([0,1]))

        # Push-Sum estimated signal eigenvector matrices
        U_s_push = v_push[:,:3]

        # Upper signal eigenvectors
        U_s_up = J_up.dot(U_s_push)

        # Lower signal eigenvectors
        U_s_down = J_down.dot(U_s_push)

        # Matrix including knowledge about DOAs of the source signals
        psi = np.linalg.inv((U_s_up.conj().T).dot(U_s_up)).dot((U_s_up.conj().T)).dot(U_s_down)

        w2, v2 = np.linalg.eig(psi)
        
        doa_1 = np.arcsin(np.angle(w2[0])/np.pi)/(2*np.pi)*360
        doa_2 = np.arcsin(np.angle(w2[1])/np.pi)/(2*np.pi)*360
        doa_3 = np.arcsin(np.angle(w2[2])/np.pi)/(2*np.pi)*360
        
        if i == 0:
            print()
            print("  DOAs of the source signals in degrees with SNR: " + str(snr_dB) )
            print("  DOAs of the source signals in degrees with N_samples: " + str(N_samples) )
            print("****************************************************************")
            print("****************************************************************")
            print("DOA of the first source signal:   " + str(doa_1))
            print("DOA of the second source signal:   " + str(doa_2))
            print("DOA of the third source signal:   " + str(doa_3))

        # Uncomment here if you want to iterate over SNR
        # MSE[snr_dB+20] = MSE[snr_dB+20] + 1/3*1/500*((doa_1-(angles[1]*360/(2*np.pi)))**2+(doa_2-(angles[0]*360/(2*np.pi)))**2+(doa_3-(angles[2]*360/(2*np.pi)))**2)
        MSE[N_samples-2] = MSE[N_samples-2] + 1/3*1/500*((doa_1-(angles[1]*360/(2*np.pi)))**2+(doa_2-(angles[0]*360/(2*np.pi)))**2+(doa_3-(angles[2]*360/(2*np.pi)))**2)

RMSE = np.sqrt(MSE)
#plt.plot(np.arange(-20,60), RMSE, 'r--')
plt.plot(np.arange(2,102), RMSE, 'r--')
plt.yscale("log")
plt.xlabel('Number of Samples N')
plt.ylabel('RMSE (degree)')
plt.show()

# load the results in memory since it takes too much time to obtain it again.(1000 iteration for N=1000)
# (Don't forget to change the name when you change some parameters above)
with open("RMSE_1000.txt", "wb") as fp:   #Pickling
    pickle.dump(RMSE, fp)



  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 2
****************************************************************
****************************************************************
DOA of the first source signal:   65.78681707020921
DOA of the second source signal:   -16.01384990210504
DOA of the third source signal:   -17.904768208985143

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 3
****************************************************************
****************************************************************
DOA of the first source signal:   35.99918252501334
DOA of the second source signal:   -18.896324731782315
DOA of the third source signal:   -26.4681330715261

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 4
****************************************************************
********************


  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 23
****************************************************************
****************************************************************
DOA of the first source signal:   -28.08199724384333
DOA of the second source signal:   -9.47373055268538
DOA of the third source signal:   -12.28424491005909

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 24
****************************************************************
****************************************************************
DOA of the first source signal:   -0.727086419204638
DOA of the second source signal:   -14.14590962869689
DOA of the third source signal:   -28.5215761526072

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 25
****************************************************************
******************


  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 44
****************************************************************
****************************************************************
DOA of the first source signal:   -29.219192980733855
DOA of the second source signal:   -13.129938674312571
DOA of the third source signal:   -5.554869177424698

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 45
****************************************************************
****************************************************************
DOA of the first source signal:   -6.5604925814459465
DOA of the second source signal:   -16.605769540455547
DOA of the third source signal:   -32.947350233365256

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 46
****************************************************************
***********


  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 65
****************************************************************
****************************************************************
DOA of the first source signal:   -31.219088250898373
DOA of the second source signal:   -7.395151762357885
DOA of the third source signal:   -15.021568409501686

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 66
****************************************************************
****************************************************************
DOA of the first source signal:   -32.43505073329068
DOA of the second source signal:   -14.615735261128174
DOA of the third source signal:   -6.7488197679022495

  DOAs of the source signals in degrees with SNR: 10
  DOAs of the source signals in degrees with N_samples: 67
****************************************************************
************